# 3D Pose Viz

In [ ]:
import pyglet
from pyglet.gl import *
from pyglet.window import key
from pyglet import shapes, text

import numpy as np
import cv2

import os
import json
import math
import yaml
from pathlib import Path
from collections import Counter

%reload_ext autoreload
%autoreload 2

## Set the env parameters..

In [ ]:
RENDER_FPS = 50
RENDER_INTERVAL = 1 / RENDER_FPS
WINDOW_WIDTH = 1920
WINDOW_HEIGHT = 1080

FILE_PATH = "./world_pose_smooth.json"

file_root, _ = os.path.splitext(FILE_PATH)
VIDEO_WRITER = cv2.VideoWriter(file_root + ".mp4", cv2.VideoWriter_fourcc(*'mp4v'), RENDER_FPS, (WINDOW_WIDTH, WINDOW_HEIGHT))

# Play the animation automatically
AUTO_PLAY = True
# Show the keystroke information
SHOW_INFO = True
# Record the sample to a video
RECORD_MODE = False


In [ ]:
pose_params = []

with open(FILE_PATH, 'r') as f:
    json_data = f.read()

data = json.loads(json_data)

for pose in data:
    keypoints = pose['data']['data']
    if len(keypoints) > 0:
        for key, value in pose['data'].items():
            print(f"{key}: {value}")
        print(len(keypoints[0]), "joints found..\n")
    

# Create OpenGL instance

In [ ]:
frame = 0

glEnable(GL_DEPTH_TEST)
glEnable(GL_LINE_SMOOTH)
camera_position = [5, -15, 5]
zoom_factor = 32.
rotation_angle_horizontal = 1.3
rotation_angle_vertical = 0.9

viewpoints = [{"zoom_factor": 32.0, "rotation_angle_horizontal": 1.3, "rotation_angle_vertical": 0.9},
              {"zoom_factor": 36.5, "rotation_angle_horizontal": 2.0, "rotation_angle_vertical": 0.3},
              {"zoom_factor": 54.0, "rotation_angle_horizontal": -0.1, "rotation_angle_vertical": -0.24},
              {"zoom_factor": 55.0, "rotation_angle_horizontal": 3.18, "rotation_angle_vertical": 0.92},
              {"zoom_factor": 22.0, "rotation_angle_horizontal": -4.72, "rotation_angle_vertical": -7.1},
              {"zoom_factor": 22.0, "rotation_angle_horizontal": -6.29, "rotation_angle_vertical": -7.5},
             ]

viewpoint = viewpoints[0]
zoom_factor, rotation_angle_horizontal, rotation_angle_vertical = viewpoint.values()


SCALE = 20
COURT_LENGTH = 5 * SCALE
COURT_WIDTH = 5 * SCALE

MAX_VERTICAL_ANGLE = math.pi / 2 - 0.1  # Just below straight up
MIN_VERTICAL_ANGLE = -MAX_VERTICAL_ANGLE  # Just below straight down

window = pyglet.window.Window(width=WINDOW_WIDTH, height=WINDOW_HEIGHT, resizable=False)


rally_label = pyglet.text.Label(
    "My 3D pose space...",
    font_name='Arial',
    font_size=20,
    x= window.width // 2,
    y= window.height - 30,
    anchor_x='center',
    anchor_y='center',
    color=(255, 255, 255, 255)
)

instructions_label = pyglet.text.Label(
    "Play/Pause: SPACE\nFrame Forward: RIGHT ARROW\nFrame Backward: LEFT ARROW\n"
    "Step Forward: SHIFT+RIGHT ARROW\nStep Backward: SHIFT+LEFT ARROW\n"
    "Record Video: R\nViewpoints: [0-5]\nClose: ESC",
    font_size=12,
    x=10,
    y=10,
    multiline=True,
    width=400,
    anchor_x="left",
    anchor_y="bottom",
)

def time_label_with_value(t):
    return text.Label(
        f"Rally Time: {t:.2f}",
        font_name='Arial',
        font_size=20,
        x=window.width - 10,
        y=10,
        multiline=True,
        width=300,
        anchor_x="right",
        anchor_y="bottom",
    )
    
def draw_surface():
    # Define the size of each square
    square_length = COURT_LENGTH / 8
    square_width = COURT_WIDTH / 8

    glColor3f(0.6, 0.6, 0.6)  # Light grey color for the grid
    glEnable(GL_BLEND)
    glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
    
    glBegin(GL_LINES)
    
    # Grid spacing based on 1/8th of the court width
    grid_spacing = COURT_WIDTH / 8
    
    max_distance = COURT_LENGTH * 2
    fade_distance = COURT_LENGTH 
    
    # Draw vertical lines
    for x in range(-int(max_distance / grid_spacing), int(max_distance / grid_spacing) + 1):
        distance = abs(x * grid_spacing)
        alpha = max(0.0, 1.0 - (distance - fade_distance) / fade_distance)
        glColor4f(0.8, 0.8, 0.8, alpha)
        glVertex3f(x * grid_spacing, -max_distance / 1, 0)
        glVertex3f(x * grid_spacing, max_distance / 1, 0)
    
    # Draw horizontal lines
    for y in range(-int(max_distance / grid_spacing), int(max_distance / grid_spacing) + 1):
        distance = abs(y * grid_spacing)
        alpha = max(0.0, 1.0 - (distance - fade_distance) / fade_distance)
        glColor4f(0.8, 0.8, 0.8, alpha)
        glVertex3f(-max_distance / 1, y * grid_spacing, 0)
        glVertex3f(max_distance / 1, y * grid_spacing, 0)
    
    glEnd()
    glDisable(GL_BLEND)



In [ ]:
# Pre-baked view points..
keystroke_to_index = {
    pyglet.window.key._0: 0,
    pyglet.window.key._1: 1,
    pyglet.window.key._2: 2,
    pyglet.window.key._3: 3,
    pyglet.window.key._4: 4,
    pyglet.window.key._5: 5
}

def on_mouse_scroll(x, y, scroll_x, scroll_y):
    global zoom_factor
    zoom_factor += scroll_y * 1.1
    zoom_factor = max(0.1, zoom_factor)  # Limit zoom factor to avoid negative values
    
def on_mouse_drag(x, y, dx, dy, buttons, modifiers):
    global rotation_angle_horizontal, rotation_angle_vertical
    rotation_speed = 0.01
    rotation_angle_horizontal += dx * rotation_speed
    rotation_angle_vertical += dy * rotation_speed
        
def on_key_press(symbol, modifiers):
    global frame
    global zoom_factor, rotation_angle_horizontal, rotation_angle_vertical
    global AUTO_PLAY
    global SHOW_INFO
    global RECORD_MODE
    
    if symbol == pyglet.window.key.LEFT:
        if modifiers & pyglet.window.key.MOD_SHIFT:
            frame = max(0, frame - RENDER_FPS)
        else:
            frame = max(0, frame - 1)

    elif symbol == pyglet.window.key.RIGHT:
        if modifiers & pyglet.window.key.MOD_SHIFT:
            frame = min(len(pose_params), frame + RENDER_FPS)
        else:
            frame = min(len(pose_params), frame + 1)
            
    elif symbol == pyglet.window.key.SPACE:
        AUTO_PLAY = not AUTO_PLAY
        if AUTO_PLAY:
            if frame >= len(pose_params):
                frame = 0
            pyglet.clock.schedule_interval(update, RENDER_INTERVAL)
            SHOW_OPTIONS_FOR_AGENT = False
        else:
            pyglet.clock.unschedule(update)

    
    elif symbol == pyglet.window.key.I:
        SHOW_INFO = not SHOW_INFO
    
    elif symbol == pyglet.window.key.R:
        RECORD_MODE = not RECORD_MODE
        if RECORD_MODE:
            # Zero the clock, enable a file write and hide the instructions.
            SHOW_INFO = False
            AUTO_PLAY = False
            frame = 0
            pyglet.clock.schedule_once(update, RENDER_INTERVAL)
    
    elif symbol == pyglet.window.key.ESCAPE or symbol == pyglet.window.key.Q:
        window.close()

    elif symbol in keystroke_to_index:
        print(symbol)
        index = keystroke_to_index[symbol]
        viewpoint = viewpoints[index]
        zoom_factor, rotation_angle_horizontal, rotation_angle_vertical = viewpoint.values()

        
def update_camera():
    global zoom_factor
    glLoadIdentity()
    
    # Update the position of the camera
    x = camera_position[0] * zoom_factor * math.cos(rotation_angle_horizontal)
    y = camera_position[1] * zoom_factor * math.sin(rotation_angle_horizontal)
    z = max(camera_position[2] * zoom_factor * -rotation_angle_vertical, 30)    

    gluLookAt(x, y, z,  # Camera position        print("Updating viewpoint:", zoom_factor, rotation_angle_horizontal, rotation_angle_vertical)
              0, 0, 0,   # Look at the origin
              0, 0, 1)   # Up vector 


@window.event
def on_draw():
    global zoom_factorpos_y
    global AUTO_PLAY
    global SHOW_OPTIONS_FOR_AGENT
    global RECORD_MODE
    global render_shot_option_flight_params

    glClearColor(0.2, 0.2, 0.2, 1)  # Dark grey background color
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(45, window.width / window.height, 10, 10000)

    glMatrixMode(GL_MODELVIEW)
    update_camera()

    # Draw the surface and labels
    draw_surface()

    # Switch to 2D mode for on-screen info
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluOrtho2D(0, window.width, 0, window.height)
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()

    # Render time, title, etc
    viz_label = pyglet.text.Label(
        "Sample path",
        font_name='Arial',
        font_size=20,
        x= window.width // 2,
        y= window.height - 30,
        anchor_x='center',
        anchor_y='center',
        color=(255, 255, 255, 255)
    )
    
    viz_label.draw()
    time_label = time_label_with_value(frame/RENDER_FPS)
    time_label.draw()

    if SHOW_INFO:
        instructions_label.draw()
                
    if RECORD_MODE:       
        # Convert the window buffer to a numpy array
        buffer = pyglet.image.get_buffer_manager().get_color_buffer()
        image_data = buffer.get_image_data()
        image_as_np = np.frombuffer(image_data.get_data(), dtype=np.uint8).reshape(WINDOW_HEIGHT, WINDOW_WIDTH, 4)
    
        # Convert RGBA to BGR (OpenCV uses BGR)
        frame_bgr = cv2.flip(cv2.cvtColor(image_as_np, cv2.COLOR_RGBA2BGR), 0)
    
        # Write frame to video
        VIDEO_WRITER.write(frame_bgr)

        pyglet.clock.schedule_once(update, RENDER_INTERVAL)

        if frame >= len(pose_params):
            # Wrap it up..
            VIDEO_WRITER.release()
            RECORD_MODE = False
        
def update(dt):
    global frame
    frame += 1

    # pyglet.clock.unschedule(update)


In [ ]:
# Event handlers..
window.on_mouse_scroll = on_mouse_scroll
window.on_mouse_drag = on_mouse_drag
window.on_draw = on_draw
window.on_key_press = on_key_press

AUTO_PLAY = False

if AUTO_PLAY:
    pyglet.clock.schedule_interval(update, RENDER_INTERVAL)
    
pyglet.app.run()

VIDEO_WRITER.release()
